<a href="https://colab.research.google.com/github/Ruiyang1210W/Stock-Analysis/blob/main/Stock_Price_Predicition3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('ASML',
                      start=start_date,
                      end=end_date,
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close",
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)

- Open: The opening price of a stock is the price at which it first trades when the market opens for that day.
- High: The highest price at which the stock traded during the day.
- Low: The lowest price at which the stock traded during the day.
- Close: The closing price is the final price at which the stock traded when the market closes for the day.
- Adj Close: The adjusted closing price reflects the closing price of the stock after taking into account any corporate actions such as dividends, stock splits, or rights offerings.
-Volume: Volume refers to the number of shares of the stock that were traded during a specific time period (typically a day).

In [ ]:
data.tail()

Price,Date,Open,High,Low,Close,Adj Close,Volume
Ticker,,ASML,ASML,ASML,ASML,ASML,ASML
3440,2024-11-29,678.869995,699.650024,678.539978,686.609985,686.609985,1706500
3441,2024-12-02,681.750000,713.679993,680.700012,711.469971,711.469971,2325900
3442,2024-12-03,702.650024,721.330017,700.619995,718.059998,718.059998,1647200
3443,2024-12-04,723.789978,724.940002,716.929993,719.919983,719.919983,1529800
3444,2024-12-05,729.150024,729.219971,708.000000,711.500000,711.500000,1799600


In [ ]:
correlation = data.corr()
sorted_correlation = correlation["Close"].iloc[:, 0].sort_values(ascending=False)
print(sorted_correlation)


Price      Ticker
Close      ASML      1.000000
Low        ASML      0.999879
Adj Close  ASML      0.999856
High       ASML      0.999822
Open       ASML      0.999643
Date                 0.888817
Volume     ASML     -0.143104
Name: ASML, dtype: float64


In [ ]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 4, 128)              │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           1,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - loss: 62644.6836
Epoch 2/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 3305.5977
Epoch 3/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 2076.0964
Epoch 4/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - loss: 1347.7930
Epoch 5/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 958.5624
Epoch 6/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 1101.5729
Epoch 7/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 1115.3965
Epoch 8/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 4242.8770
Epoch 9/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 1212.3658
Epoch 10/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - loss: 1701.6759
Epoch 11/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 1643.8331
Epoch 12/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 1398.9404
Epoch 13/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - loss: 1617.4342
Epoch 14/30
2756/2756 ━━━━━━━━━━━━━━━━━━━━ 19s 

In [ ]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[729.15, 729.22, 708,711.5, 1799600]])
model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step


array([[691.9857]], dtype=float32)

I change tickers and retrain the model one by one, so the output above only reflects the prediction for the most recent ticker.

- The model predict the close price of AAPL is 226.66386.
which is lower then the current price: 242.65
Since  I bought in 229.37. I will **hold** the AAPL share this week.

- The model predict the close price of AMZN is 198.0678
which is lower then the current price: $218.94. I bought in 197
So I will **hold** the AMZN share this week to wait untill price get higher.  The stock went up is possibly because of the Holiday Season.

- The model predict the close price of TSLA is 314.31226
which is lower then the current price: 357.93. I bought in 352.54
So I will **hold** the TSLA share this week to wait untill price get higher.
- The model predict the close price of ASML is 691.9
which is  much lower then the current price: 711.5. I will wait until the price drop to buy it.

- The model predict the close price of NVDA is 144.109. The current price is 145.06. This slight difference suggests the model's prediction aligns closely with the current market valuation. I believe investing in NVDA could be a viable option. Conclusion: I will **buy** it this week.
- The model predict the close price of GOOG is 183.4969, however, the current price is 177.32. The prediction is clearly higher then the current price, so I will **buy** it this week. ( I bought 2 trades for google)